In [1]:
# This code here is intended to create a custom keras layer.
# This layer takes as input the path to the image
# It outputs an array that can be given as an input to a convolutional neural network

In [23]:
#Create constants and import libraries
X_DIM = 256
Y_DIM = 256
import os
import numpy as np
from PIL import Image as imLib
from keras.layers import Layer
from keras import models
from keras import layers
from keras.utils import Sequence
from math import ceil
import time


In [24]:
#Build custom layer

class ImageBatchSequence(Sequence):
    def __init__(self, x_set, batch_size, x_dim = X_DIM, y_dim = Y_DIM):
        self.x = x_set
        y_set = [(1 if "fake" in x else 0) for x in x_set] #This makes the output to be 1 (AI-generated) if FAKE is in pathname.
        #                                                  # Otherwise, it makes the output 0 (not AI-generated)
        self.y = y_set
        self.batch_size = batch_size
        self.x_dim = x_dim
        self.y_dim = y_dim
    def __len__(self):
        return ceil(len(self.x)/self.batch_size)
    def __getitem__(self, index):
        low = index * self.batch_size
        high = min(low+self.batch_size, len(self.x))
        batch_x = self.x[low : high]
        batch_y = self.y[low : high]
        image_x_batch = []
        for imPath in batch_x:
            image = np.asarray(imLib.open(imPath)).reshape(self.x_dim,self.y_dim,3)
            image_x_batch.append(image)
        return np.array(image_x_batch), np.array(batch_y)
